In [1]:
import pandas as pd
from pathlib import Path
import torch
from torch.utils.data import Dataset, DataLoader

In [2]:
data_path = Path('data')

In [3]:
reranked_results_alibaba = pd.read_parquet(data_path / 'reranked_results_alibaba_finetuned.parquet')
reranked_results_alibaba

,post_id,tweet_text,cord_uid,bm25_topk,reranked_topk,reranked_docs,reranked_scores
0,16,covid recovery: this study from the usa reveal...,3qvh482o,"[atji1xge, mb18fj8a, 66g5lpm6, 59up4v56, gatxu...","[8t2tic9n, 00ia8k0b, 6mfd3n4s, o4vvlmr4, 59up4...","[8t2tic9n, 00ia8k0b, 6mfd3n4s, o4vvlmr4, 59up4...","[2.205078125, 2.12109375, 2.03515625, 1.970703..."
1,69,"""Among 139 clients exposed to two symptomatic ...",r58aohnu,"[r58aohnu, p0kg6dyz, yrowv62k, s2vckt2w, j1ucr...","[r58aohnu, yrowv62k, eay6qfhz, iu1d9i57, kiq6x...","[r58aohnu, yrowv62k, eay6qfhz, iu1d9i57, kiq6x...","[2.978515625, 1.8056640625, 1.66796875, 1.2822..."
2,73,I recall early on reading that researchers who...,sts48u9i,"[sgo76prc, sts48u9i, tz2shoso, gruir7aw, 3xw4q...","[sts48u9i, 1uto8vrp, geo7ac5i, gruir7aw, 3xw4q...","[sts48u9i, 1uto8vrp, geo7ac5i, gruir7aw, 3xw4q...","[2.451171875, 1.625, 1.619140625, 1.53125, 1.5..."
3,93,You know you're credible when NIH website has ...,3sr2exq9,"[3sr2exq9, hgpiig0g, sv48gjkk, 1cpjqav4, k0f4c...","[3sr2exq9, k0f4cwig, 8j3bb6zx, sv48gjkk, kdegn...","[3sr2exq9, k0f4cwig, 8j3bb6zx, sv48gjkk, kdegn...","[3.298828125, 2.234375, 1.8798828125, 1.763671..."
4,96,Resistance to antifungal medications is a grow...,ybwwmyqy,"[3l6ipiwk, vabb2f26, ouvq2wpq, lzddnb8j, ybwwm...","[ybwwmyqy, ouvq2wpq, rs3umc1x, 3l6ipiwk, lzddn...","[ybwwmyqy, ouvq2wpq, rs3umc1x, 3l6ipiwk, lzddn...","[1.9306640625, 1.75, 1.748046875, 1.6279296875..."
...,...,...,...,...,...,...,...
14248,14248,"""evidence on covid-19 reveals a growing body o...",9169o29b,"[qfrenkb6, dgng39yd, l9lni5d3, 1y1ik2u9, jgq96...","[jgq968f6, 1y1ik2u9, dgng39yd, s2hp3sat, 4p6fc...","[jgq968f6, 1y1ik2u9, dgng39yd, s2hp3sat, 4p6fc...","[2.626953125, 2.5546875, 2.521484375, 2.515625..."
14249,14249,Outdoor lighting has detrimental impacts on lo...,s2bpha8l,"[s2bpha8l, 8a3fp7ym, wtovv1jy, 3a7utmsd, 2u9ee...","[s2bpha8l, 8a3fp7ym, k8v832rl, jkdyo2zp, 3a7ut...","[s2bpha8l, 8a3fp7ym, k8v832rl, jkdyo2zp, 3a7ut...","[2.919921875, 1.90234375, 0.95703125, 0.311767..."
14250,14250,"26/ and influenza virus (and other pathogens, ...",atloc9th,"[2d2y5gmg, lavcsqov, 7s79900j, y7s6qt6j, p3qcf...","[7s79900j, 7y6ok9a2, e0pbs354, 848jmwgc, wdvuw...","[7s79900j, 7y6ok9a2, e0pbs354, 848jmwgc, wdvuw...","[1.513671875, 1.369140625, 1.30859375, 1.21386..."
14251,14251,does it?'sars-cov-2-naïve vaccinees had a 13.0...,t4y1ylb3,"[7a543f7v, t4y1ylb3, a4klrp3h, lcy7u1i6, sjsaw...","[7a543f7v, t4y1ylb3, ws3tfs1j, 0o7rm667, ro3k6...","[7a543f7v, t4y1ylb3, ws3tfs1j, 0o7rm667, ro3k6...","[3.78515625, 3.7578125, 2.0859375, 1.892578125..."


In [4]:
reranked_results_tinybert = pd.read_parquet(data_path / 'reranked_results_tinyBERT_finetuned.parquet')
reranked_results_tinybert

,post_id,tweet_text,cord_uid,bm25_topk,reranked_topk,reranked_docs,reranked_scores,in_topx
0,16,covid recovery: this study from the usa reveal...,3qvh482o,"[atji1xge, mb18fj8a, 66g5lpm6, 59up4v56, gatxu...","[atji1xge, 8t2tic9n, 59up4v56, o4vvlmr4, styav...","[atji1xge, 8t2tic9n, 59up4v56, o4vvlmr4, styav...","[2.712890625, 2.529296875, 2.423828125, 1.1914...",0.0
1,69,"""Among 139 clients exposed to two symptomatic ...",r58aohnu,"[r58aohnu, p0kg6dyz, yrowv62k, s2vckt2w, j1ucr...","[r58aohnu, 6zfpcm4j, 8n4zf9oo, zd8c1no7, kiq6x...","[r58aohnu, 6zfpcm4j, 8n4zf9oo, zd8c1no7, kiq6x...","[3.83984375, -0.86181640625, -0.86962890625, -...",1.0
2,73,I recall early on reading that researchers who...,sts48u9i,"[sgo76prc, sts48u9i, tz2shoso, gruir7aw, 3xw4q...","[sts48u9i, o47v5vgw, gruir7aw, az7f7zgr, cgc0v...","[sts48u9i, o47v5vgw, gruir7aw, az7f7zgr, cgc0v...","[0.96044921875, 0.75927734375, 0.7197265625, 0...",1.0
3,93,You know you're credible when NIH website has ...,3sr2exq9,"[3sr2exq9, hgpiig0g, sv48gjkk, 1cpjqav4, k0f4c...","[3sr2exq9, k0f4cwig, sv48gjkk, 8j3bb6zx, kdegn...","[3sr2exq9, k0f4cwig, sv48gjkk, 8j3bb6zx, kdegn...","[3.14453125, 1.6796875, 0.9404296875, 0.323730...",1.0
4,96,Resistance to antifungal medications is a grow...,ybwwmyqy,"[3l6ipiwk, vabb2f26, ouvq2wpq, lzddnb8j, ybwwm...","[ybwwmyqy, vabb2f26, sxx3yid9, ouvq2wpq, 3l6ip...","[ybwwmyqy, vabb2f26, sxx3yid9, ouvq2wpq, 3l6ip...","[4.60546875, 3.138671875, 2.51953125, 2.341796...",1.0
...,...,...,...,...,...,...,...,...
14248,14248,"""evidence on covid-19 reveals a growing body o...",9169o29b,"[qfrenkb6, dgng39yd, l9lni5d3, 1y1ik2u9, jgq96...","[jgq968f6, oywhabcd, dgng39yd, xs7vkm19, 9169o...","[jgq968f6, oywhabcd, dgng39yd, xs7vkm19, 9169o...","[2.265625, 1.3544921875, 0.8896484375, 0.71289...",0.2
14249,14249,Outdoor lighting has detrimental impacts on lo...,s2bpha8l,"[s2bpha8l, 8a3fp7ym, wtovv1jy, 3a7utmsd, 2u9ee...","[s2bpha8l, 8a3fp7ym, jkdyo2zp, k8v832rl, t3giv...","[s2bpha8l, 8a3fp7ym, jkdyo2zp, k8v832rl, t3giv...","[9.3984375, 4.375, -0.5166015625, -0.708007812...",1.0
14250,14250,"26/ and influenza virus (and other pathogens, ...",atloc9th,"[2d2y5gmg, lavcsqov, 7s79900j, y7s6qt6j, p3qcf...","[y7s6qt6j, 2d2y5gmg, ahpkmffy, 848jmwgc, asj73...","[y7s6qt6j, 2d2y5gmg, ahpkmffy, 848jmwgc, asj73...","[3.01953125, 1.8671875, 0.78515625, 0.72070312...",0.0
14251,14251,does it?'sars-cov-2-naïve vaccinees had a 13.0...,t4y1ylb3,"[7a543f7v, t4y1ylb3, a4klrp3h, lcy7u1i6, sjsaw...","[7a543f7v, t4y1ylb3, 5vx2ylui, o86wki37, zqjad...","[7a543f7v, t4y1ylb3, 5vx2ylui, o86wki37, zqjad...","[5.5703125, 5.40625, 4.49609375, 1.83984375, 1...",0.5


In [5]:
# Merge reranker results
reranked_results = pd.concat([reranked_results_alibaba, reranked_results_tinybert], keys=['alibaba', 'tinybert'], names=['reranker'])
reranked_results = reranked_results.reset_index(level='reranker').reset_index(drop=True)

In [6]:
relevance_labels = reranked_results_alibaba[['post_id', 'cord_uid']].set_index('post_id').to_dict()['cord_uid']

In [7]:
# calc rrf with rank and with score?

# rank

# df with post_id, gt, results for every reranker
# merge dataframes on post_id only keep post_id, cord_uid, reranked_topk using the reranker name as suffix

dfs = [reranked_results_alibaba[['post_id', 'cord_uid', 'reranked_topk']].rename(columns={'reranked_topk': 'reranked_topk_alibaba'}),
        reranked_results_tinybert[['post_id', 'cord_uid', 'reranked_topk']].rename(columns={'reranked_topk': 'reranked_topk_tinybert'})]
merged_df = dfs[0]
merged_df
for df in dfs[1:]:
    # add reranked_topk column to merged_df
    merged_df = merged_df.merge(df.drop(columns=['cord_uid']), on='post_id', how='inner')

merged_df



,post_id,cord_uid,reranked_topk_alibaba,reranked_topk_tinybert
0,16,3qvh482o,"[8t2tic9n, 00ia8k0b, 6mfd3n4s, o4vvlmr4, 59up4...","[atji1xge, 8t2tic9n, 59up4v56, o4vvlmr4, styav..."
1,69,r58aohnu,"[r58aohnu, yrowv62k, eay6qfhz, iu1d9i57, kiq6x...","[r58aohnu, 6zfpcm4j, 8n4zf9oo, zd8c1no7, kiq6x..."
2,73,sts48u9i,"[sts48u9i, 1uto8vrp, geo7ac5i, gruir7aw, 3xw4q...","[sts48u9i, o47v5vgw, gruir7aw, az7f7zgr, cgc0v..."
3,93,3sr2exq9,"[3sr2exq9, k0f4cwig, 8j3bb6zx, sv48gjkk, kdegn...","[3sr2exq9, k0f4cwig, sv48gjkk, 8j3bb6zx, kdegn..."
4,96,ybwwmyqy,"[ybwwmyqy, ouvq2wpq, rs3umc1x, 3l6ipiwk, lzddn...","[ybwwmyqy, vabb2f26, sxx3yid9, ouvq2wpq, 3l6ip..."
...,...,...,...,...
14248,14248,9169o29b,"[jgq968f6, 1y1ik2u9, dgng39yd, s2hp3sat, 4p6fc...","[jgq968f6, oywhabcd, dgng39yd, xs7vkm19, 9169o..."
14249,14249,s2bpha8l,"[s2bpha8l, 8a3fp7ym, k8v832rl, jkdyo2zp, 3a7ut...","[s2bpha8l, 8a3fp7ym, jkdyo2zp, k8v832rl, t3giv..."
14250,14250,atloc9th,"[7s79900j, 7y6ok9a2, e0pbs354, 848jmwgc, wdvuw...","[y7s6qt6j, 2d2y5gmg, ahpkmffy, 848jmwgc, asj73..."
14251,14251,t4y1ylb3,"[7a543f7v, t4y1ylb3, ws3tfs1j, 0o7rm667, ro3k6...","[7a543f7v, t4y1ylb3, 5vx2ylui, o86wki37, zqjad..."


In [8]:
merged_df.columns[2:]  

Index(['reranked_topk_alibaba', 'reranked_topk_tinybert'], dtype='object')

In [9]:
# for each document in set of reraked documents calculate RRF, if 

# for each row on merged df, get set of reranked documents across all rerankers, for each document in set of reranked documents s


def calculate_rrf_scores(row, reranker_columns, k=60):
    reranked_docs = set()
    for col in reranker_columns:
        reranked_docs.update(row[col])
    
    scores = {doc: 0 for doc in reranked_docs}
    for col in reranker_columns:
        for doc in row[col]:
            rank = list(row[col]).index(doc) + 1
            scores[doc] += 1 / (rank + k)
    
    return scores
                            
def get_final_ranking(row, scores):
    return sorted(scores.keys(), key=lambda x: scores[x], reverse=True)

# Define reranker columns
reranker_columns = merged_df.columns[1:]

# Calculate RRF scores and final ranking for each row
merged_df['rrf_scores'] = merged_df.apply(lambda row: calculate_rrf_scores(row, reranker_columns), axis=1)
merged_df['final_ranking'] = merged_df.apply(lambda row: get_final_ranking(row, row['rrf_scores']), axis=1)

In [10]:
# check if cord_uid is in the top 3 of final ranking
def check_cord_uid_in_final_ranking(row, k=3):
    # Check if the cord_uid is in the top k final ranking documents
    return row['cord_uid'] in row['final_ranking'][:k]

# Apply the function to the DataFrame
merged_df['in_top_3_final'] = merged_df.apply(check_cord_uid_in_final_ranking, axis=1)
# print average
print("Average of cord_uid in top 3 for final ranking:", merged_df['in_top_3_final'].mean())
# check if cord_uid is in the top 5 of final ranking

Average of cord_uid in top 3 for final ranking: 0.727495965761594


In [11]:
# check how often cord_uid is in the top 3 of reranked_topk_aliababa 
def check_cord_uid_in_top_k(row, k=3):
    # Check if the cord_uid is in the top k reranked documents
    return row['cord_uid'] in row['reranked_topk_alibaba'][:k]

# Apply the function to the DataFrame
merged_df['in_top_3_alibaba'] = merged_df.apply(check_cord_uid_in_top_k, axis=1)
# print average
print("Average of cord_uid in top 3 for alibaba:", merged_df['in_top_3_alibaba'].mean())

Average of cord_uid in top 3 for alibaba: 0.757244088963727


In [12]:
# check how often cord_uid is in the top 3 of tinybert
def check_cord_uid_in_top_k(row, k=3):
    # Check if the cord_uid is in the top k reranked documents
    return row['cord_uid'] in row['reranked_topk_tinybert'][:k]

# Apply the function to the DataFrame
merged_df['in_top_3_tinybert'] = merged_df.apply(check_cord_uid_in_top_k, axis=1)
# print average
print("Average of cord_uid in top 3 for tinybert:", merged_df['in_top_3_tinybert'].mean())

Average of cord_uid in top 3 for tinybert: 0.6873640637058864
